## 로컬 환경에서 PDF 파일 RAG 검색하기 3단계 
### - 사용한 임베딩 모델 : jhgan/ko-sroberta-multitask
### - 사용한 LLM 모델 : llama3.2

__step2__
- PDF 문서 여러개 로드 (data 폴더에 있는 문서 전부 로드)
- 문서를 임베딩하여 csv 파일로 저장 (저장 경로 : csv 폴더)
- csv 파일을 FAISS 인덱싱 : 결과물이 인덱스로 나옴
- FAISS 인덱스를 파일로 만들어 디스크에 저장

- FAISS 인덱스와 랭체인 FAISS 인덱스는 구분됨
- csv 파일로 랭체인 FAISS 인덱스 생성하여 검색하기 까지 구현

In [12]:
# 필요한 라이브러리 임포트
from langchain.document_loaders import DirectoryLoader,PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain_core.embeddings import Embeddings  
from sentence_transformers import SentenceTransformer

import csv
import faiss
import numpy as np
import pandas as pd
import os

# 1. 문서 로드
# data 폴더 안에 있는 pdf 파일 전부 로드하기
loader = DirectoryLoader(
    'data',
    glob='*.pdf',
    loader_cls=PyMuPDFLoader
)
docs = loader.load()

# 2. 문서 분할
# 텍스트를 1000자 단위로 나눔 (chunk size), 각 청크 간 50자씩 겹치도록 설정
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)

# 3. 임베딩을 하기 위한 클래스 생성
class KoSentenceTransformerEmbeddings(Embeddings):
    # 임베딩 모델 초기화 
    def __init__(self, model_name):
        self.model = SentenceTransformer(model_name)
    # 여러개의 문서를 임베딩하여 벡터 데이터 생성 , 벡터화 된 각 문서가 리스트 형태로 반환 
    def embed_documents(self, texts):
        return self.model.encode(texts, convert_to_numpy=True).tolist()
    # 검색 쿼리를 벡터화 
    def embed_query(self, text):
        return self.model.encode([text], convert_to_numpy=True).tolist()[0]
    
# 3.1. 임베드 모델 로딩
embedding_model = KoSentenceTransformerEmbeddings("jhgan/ko-sroberta-multitask")

# 3.2 문서를 임베딩 하여 csv 파일로 저장하기 위한 함수 생성
def save_embeddings_to_csv(documents, embedding_model, file_path):
    os.makedirs(file_path, exist_ok=True)

    file_docs = {}
    for doc in documents:
        file_name = os.path.basename(doc.metadata['source']).replace('.pdf','')
        if file_name not in file_docs:
            file_docs[file_name] = []
        file_docs[file_name].append(doc)
    
    for file_name, docs in file_docs.items():
        full_path = os.path.join(file_path, f"{file_name}.csv")

        #  임베딩 
        embeddings = embedding_model.embed_documents([doc.page_content for doc in docs])
        #  임베딩 결과를 CSV 로 저장
        with open(full_path, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(["document", "embedding"])
            
            for doc, embedding in zip(docs, embeddings):
                writer.writerow([doc.page_content, embedding])
        
        print(f"임베딩 데이터가 {full_path} 파일에 저장되었습니다.")       
    
# 3.3 함수 실행 하여 CSV 파일 생성
save_embeddings_to_csv(split_documents, embedding_model, 'csv/')


임베딩 데이터가 csv/SPRI_AI_Brief_2023년12월호_F.csv 파일에 저장되었습니다.
임베딩 데이터가 csv/AI기반_인파분석플랫폼구축_제안서.csv 파일에 저장되었습니다.
임베딩 데이터가 csv/운영체제_중간과제물.csv 파일에 저장되었습니다.


In [19]:
import os
import ast
import numpy as np
import pandas as pd
from langchain.schema import Document
from langchain.vectorstores import FAISS

# 📂 데이터 폴더 (CSV 파일이 있는 위치)
data_folder = "./csv"

# 💾 FAISS 인덱스를 저장할 폴더
faiss_index_folder = "./faiss_index"

# CSV에서 문서 + 벡터 로드
def load_csv_embeddings(folder_path):
    documents = []
    embeddings = []
    metadatas = []

    for filename in os.listdir(folder_path):
        if filename.endswith(".csv"):
            file_path = os.path.join(folder_path, filename)
            df = pd.read_csv(file_path)

            # 🛠️ CSV에서 `text`, `embedding` 컬럼이 있는지 확인
            if "document" in df.columns and "embedding" in df.columns:
                for _, row in df.iterrows():
                    text = row["document"]
                    
                    # 문자열을 리스트로 변환 (예: "[0.1, 0.2, ...]" → [0.1, 0.2, ...])
                    embedding = ast.literal_eval(row["embedding"])

                    # 변환된 데이터 저장
                    documents.append(text)  # 문서 내용만 저장
                    embeddings.append(np.array(embedding, dtype=np.float32))
                    metadatas.append({"source": filename})  # 메타데이터 추가
            else:
                print(f"⚠️ {filename} 파일에 'text' 또는 'embedding' 컬럼이 없습니다!")

    return documents, embeddings, metadatas

# FAISS 인덱스 생성 및 저장
def create_faiss_index(embeddings, documents, metadatas, save_path):
    # FAISS 벡터스토어 생성
    vector_store = FAISS.from_embeddings(embeddings, metadatas)

    # 인덱스 저장
    os.makedirs(save_path, exist_ok=True)
    faiss_index_path = os.path.join(save_path, "faiss_index")
    vector_store.save_local(faiss_index_path)

    print(f"✅ FAISS 인덱스 저장 완료: {faiss_index_path}")

# 실행
if __name__ == "__main__":
    # 1️⃣ CSV에서 문서 + 벡터 로드
    documents, embeddings, metadatas = load_csv_embeddings(data_folder)
    print(f"📄 {len(documents)}개의 문서 로드 완료!")


📄 2개의 문서 로드 완료!


In [ ]:

    # 2️⃣ FAISS 인덱스 생성 및 저장
    create_faiss_index(embeddings, documents, metadatas, faiss_index_folder)


In [15]:
# 4. 임베딩 데이터를 랭체인 FAISS 로 인덱싱 하는 과정
import os
import ast
import pandas as pd
import numpy as np
from langchain.schema import Document
from langchain.vectorstores import FAISS

# 📂 데이터 폴더 (CSV가 저장된 위치)
data_folder = "./csv"

# 💾 FAISS 인덱스를 저장할 폴더
faiss_index_folder = "./faiss_index"


# 4.1. CSV 파일에서 문서와 벡터 데이터를 로드 
def load_csv_embeddings(folder_path):
    documents = []
    embeddings = []
    
    for filename in os.listdir(folder_path):
        if filename.endswith(".csv"):
            file_path = os.path.join(folder_path, filename)
            df = pd.read_csv(file_path)

            # 🛠️ CSV에서 `text`, `embedding` 컬럼이 있다고 가정
            if "document" in df.columns and "embedding" in df.columns:
                for _, row in df.iterrows():
                    text = row["document"]
                    
                    # 문자열로 저장된 리스트를 실제 리스트로 변환 (예: "[0.1, 0.2, ...]" → [0.1, 0.2, ...])
                    embedding = ast.literal_eval(row["embedding"])

                    # 변환된 데이터 저장
                    documents.append(Document(page_content=text))
                    embeddings.append(np.array(embedding, dtype=np.float32))
            else:
                print(f"⚠️ {filename} 파일에 'text' 또는 'embedding' 컬럼이 없습니다!")

    return documents, embeddings

# 4.2. FAISS 인덱스를 생성하여 저장
def create_faiss_index(documents, embeddings, save_path):
    # FAISS 벡터스토어 생성
    vector_store = FAISS.from_embeddings(embeddings, documents)

    # 인덱스 저장
    os.makedirs(save_path, exist_ok=True)
    faiss_index_path = os.path.join(save_path, "faiss_index")
    vector_store.save_local(faiss_index_path)
    
    print(f"✅ FAISS 인덱스 저장 완료: {faiss_index_path}")

# 4.3. 함수 실행
if __name__ == "__main__":
    # 1️⃣ CSV에서 문서 + 벡터 로드
    documents, embeddings = load_csv_embeddings(data_folder)
    print(f"📄 {len(documents)}개의 문서 로드 완료!")

    # 2️⃣ FAISS 인덱스 생성 및 저장
    create_faiss_index(documents, embeddings, faiss_index_folder)

📄 133개의 문서 로드 완료!


ValueError: too many values to unpack (expected 2)